### A first API

[Chronicling America](http://chroniclingamerica.loc.gov/about/) is a joint project of the National Endowment for the Humanities and the Library of Congress .

Search for articles that mention "[slavery](http://chroniclingamerica.loc.gov/search/pages/results/?andtext=slavery)".

<div class="alert alert-info">

Look at the URL. What happens if you change the word slavery to abolition? 

What happens to the URL when you go to the second page? Can you get to page 251?

</div>

What if we append ``&format=json`` to the end of the search URL? 


http://chroniclingamerica.loc.gov/search/pages/results/?andtext=slavery&format=json


In [ ]:
import requests

In [ ]:
base_url   = 'http://chroniclingamerica.loc.gov/search/pages/results/'
parameters = '?andtext=slavery&format=json'

In [ ]:
r = requests.get(base_url + parameters)

In [ ]:
search_json = r.json()

In [ ]:
print search_json.keys()

In [ ]:
search_json['totalItems']

<div class="alert alert-info">
What else is in there? Where is the stuff we want?
</div>

In [ ]:
first_item = search_json['items'][0]

print first_item.keys()

<div class="alert alert-info">
What is the title of the first item?
</div>

In [ ]:
import pandas as pd

# Make sure all columns are displayed
pd.set_option("display.max_columns",101)

In [ ]:
df = pd.DataFrame(search_json['items'])

df.head(6)

In [ ]:
pd.DataFrame(search_json)

In [ ]:
df.to_csv('lynching_articles.csv')

<div class="alert alert-info">
Look at the API guidelines. How can we get the third page?
</div>

In [ ]:
base_url   = 'http://chroniclingamerica.loc.gov/search/pages/results/'

### edit this line
parameters = '?andtext=slavery&format=json'


r = requests.get(base_url + parameters)
results =  r.json()

print results['startIndex']
print results['endIndex']

In [ ]:
base_url = 'http://chroniclingamerica.loc.gov/search/pages/results/'
parameters = {'andtext': 'lynching',
              'page' : 3,
              'format'  : 'json'}
r = requests.get(base_url, params=parameters)

results =  r.json()

print results['startIndex']
print results['endIndex']

In [ ]:
def get_articles():
    '''
    Make calls to the Chronicling America API.
    '''
    
    base_url   = 'http://chroniclingamerica.loc.gov/search/pages/results/'
    parameters = {'andtext': 'lynching',
                  'page'   : 3,
                  'format' : 'json'}
    
    r = requests.get(base_url, params = parameters)
    results =  r.json()
    
    return results

In [ ]:
results = get_articles()

print results['startIndex']
print results['endIndex']

In [ ]:
def get_articles(search_term, page_number):
    '''
    Make calls to the Chronicling America API.
    '''
    
    base_url = 'http://chroniclingamerica.loc.gov/search/pages/results/'
    parameters = {'andtext': search_term,
                  'page'   : page_number,
                  'format' : 'json'}
    
    r = requests.get(base_url, params = parameters)
    results =  r.json()

    return results

In [ ]:
results = get_articles('lynching', 3)

print results['startIndex']
print results['endIndex']

In [ ]:
for page_number in range(1,4): # range stops before it gets to the last number
    results = get_articles('lynching', page_number)
    print results['startIndex'], results['endIndex']
    

In [ ]:
df = pd.DataFrame() # empty dataframe to store results

for page_number in range(1,4):
    results = get_articles('lynching', page_number)
    new_df = pd.DataFrame(results['items'])
    df = df.append(new_df , ignore_index=True) # otherwise, index would be 0-20 3x
    
print len(df)
df.head(5)

<div class="alert alert-info">
<h1> Geocoding</h1>

Work in groups of three!
<p>
You have been handed a list of addresses. You want to geocode them. 
<p>Read about the Google Maps Geocoding API.
<p>On paper, map out your work flow. What functions will you need?

</div>

File location:

https://raw.githubusercontent.com/nealcaren/CSSS-CABD/master/files/locations.csv

<div class="alert alert-info">

My workflow:   
   
   
1. Use `requests` to test out a single address.    
2. Turn that into a function that accepts a location.    
3. Read in the CSV file with all the locations.    
4. Store the results    
5. Write them to a CSV.    
</div>

In [ ]:
import oauth2

In [ ]:
consumer_key    = 'qDBPo9c_szHVrZwxzo-zDw'
consumer_secret = '4we8Jz9rq5J3j15Z5yCUqmgDJjM'
token           = 'jeRrhRey_k-emvC_VFLGrlVHrkR4P3UF'
token_secret    = 'n-7xHNCxxedmAMYZPQtnh1hd7lI'

consumer = oauth2.Consumer(consumer_key, consumer_secret)

In [ ]:
category_filter = 'restaurants'
location        = 'Oslo, Norway'

options         =  'category_filter=%s&location=%s&sort=1' % (category_filter, location)
url             = 'http://api.yelp.com/v2/search?' + options

oauth_request = oauth2.Request('GET', url, {})

oauth_request.update({'oauth_nonce'      : oauth2.generate_nonce(),
                      'oauth_timestamp'  : oauth2.generate_timestamp(),
                      'oauth_token'       : token,
                      'oauth_consumer_key': consumer_key})

token = oauth2.Token(token, token_secret)
oauth_request.sign_request(oauth2.SignatureMethod_HMAC_SHA1(), consumer, token)
signed_url = oauth_request.to_url()

print signed_url

In [ ]:
resp = requests.get(url=signed_url)
oslo_restaurants = resp.json()

print oslo_restaurants.keys()

In [ ]:
oslo_restaurants['businesses'][1]

In [ ]:
print oslo_restaurants['total']
print len(oslo_restaurants['businesses'])

In [ ]:
def get_yelp_page(location, offset):
    '''
    Retrieve one page of results from the Yelp API
    Returns a JSON file
    '''
    # from https://github.com/Yelp/yelp-api/tree/master/v2/python
    consumer_key    = 'qDBPo9c_szHVrZwxzo-zDw'
    consumer_secret = '4we8Jz9rq5J3j15Z5yCUqmgDJjM'
    token           = 'jeRrhRey_k-emvC_VFLGrlVHrkR4P3UF'
    token_secret    = 'n-7xHNCxxedmAMYZPQtnh1hd7lI'
    
    consumer = oauth2.Consumer(consumer_key, consumer_secret)
    
    url = 'http://api.yelp.com/v2/search?category_filter=restaurants&location=%s&sort=1&offset=%s' % (location, offset)
    
    oauth_request = oauth2.Request('GET', url, {})
    oauth_request.update({'oauth_nonce': oauth2.generate_nonce(),
                          'oauth_timestamp': oauth2.generate_timestamp(),
                          'oauth_token': token,
                          'oauth_consumer_key': consumer_key})
    
    token = oauth2.Token(token, token_secret)
    
    oauth_request.sign_request(oauth2.SignatureMethod_HMAC_SHA1(), consumer, token)
    
    signed_url = oauth_request.to_url()
    resp = requests.get(url=signed_url)
    return resp.json()

def get_yelp_results(location):
    '''
    Retrive both pages of results from the Yelp API
    Returns a dataframe
    '''
    df = pd.DataFrame()
    for offset in [1,21]:
        results = get_yelp_page(location, offset)
        new_df = pd.DataFrame(results['businesses'])
        df = df.append(new_df , ignore_index=True)
    return df

In [ ]:
ch_df = get_yelp_results('Chapel Hill, NC')

print len(ch_df)

In [ ]:
ch_df.keys()

In [ ]:
ch_df[['name','categories','review_count','rating']].sort_values(by='rating', ascending=False)

In [ ]:
oslo_df = get_yelp_results('Oslo, Norway')
oslo_df[['name','categories','review_count','rating']].sort_values(by='rating', ascending=False)


<div class="alert alert-info">
Your turn. Modify the function to take different kinds of business. 

You can also add a category of business to search for from the [list](https://www.yelp.com/developers/documentation/v2/category_list) of acceptable values. 
</div>

<div class="alert alert-info">
Your turn. Sign up to be a developer on Twitter. Figure out the next steps...
</div>